<a href="https://colab.research.google.com/github/csetapan/MyResearchFile/blob/master/Preprocessing_COVID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import imageio
import matplotlib.pyplot as plt

In [ ]:
img1 = cv2.imread("CT_COVID.png")
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img1 = img1[:,:,0]

plt.imshow(img,cmap='gray')
plt.axis('off')
plt.show();

In [ ]:
import cv2
img = cv2.resize(img,(200,200))
img = img[:,:,0]
plt.hist(img.ravel(),bins=256)

In [ ]:
#img2 = cv2.imread("CT_COVID_fig2.png")
#img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
#plt.hist(img2.ravel(),bins=256);

In [ ]:
#img3 = cv2.imread("CT_COVID_fig3.png")
#img3 = cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
#plt.hist(img3.ravel(),bins=256);

In [ ]:
#img4 = cv2.imread("CT_COVID_fig4.png")
#img4 = cv2.cvtColor(img4,cv2.COLOR_BGR2GRAY)
#plt.hist(img4.ravel(),bins=256);

In [ ]:
ret,thresh = cv2.threshold(img,0,255,cv2.THRESH_BINARY)
img_binary = img<thresh
plt.imshow(img_binary,cmap='gray')
plt.axis('off')
plt.show();

In [ ]:
from skimage.filters import try_all_threshold    
fig,ax = try_all_threshold(img,verbose=False)
#plt.imshow(fig,cmap='gray')
plt.show();

In [ ]:
from skimage.filters import threshold_mean

thresh = threshold_mean(img)
img_binary = img<thresh
plt.imshow(img_binary,cmap='gray')
plt.axis('off')
plt.show();

In [ ]:
img1 = 255-img
plt.imshow(img1,cmap='gray')
plt.show();

In [ ]:
import numpy as np
img1 = np.where(img_binary>0,img,0)
plt.imshow(img1,cmap='gray')
plt.show();

In [ ]:
img2=255.0-img1
plt.imshow(img1,cmap='gray')
plt.show();

In [ ]:
image1 = img2[55:130,35:85]
image2 = img2[95:125,153:170]

fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))

ax1.imshow(image1,cmap='jet')
ax2.imshow(image2,cmap='jet')

In [ ]:
# Also wattershed  is used for segmentation using image boundary
#Whwn many objects are very close to each other watershed technique is used to distingush the boundary
# In watershed , we consider the gray image , wher each high intensites are considered as peaks and the low intensities are valley
#Water of different color are poured in different valey and it rises to touch the peak, hence the water of different valley will
# touch each other, this is considered as the bounary positions

In [ ]:
from __future__ import division,print_function
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure,color,io
from skimage.segmentation import clear_border
#from scipy.ndimage import distance_transform_edt

kernel = np.ones((5,5),np.uint8)

#threshold value of the image by using otsu + binary threshold
ret,thresh = cv2.threshold(img,0,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

#MORPHOLOGICAL OPERATION TO REMOVE SMALL NOISE = opening
#To remove holes we have used = closing
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel,iterations=2)

#opening = clear_border(opening)

Back_ground = cv2.erode(opening,kernel,iterations=1)
plt.imshow(Back_ground,cmap='gray')
#plt.title("Background")
plt.axis('off')
plt.show();

distance_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)

ret2,Fore_ground = cv2.threshold(distance_transform,0.0999*distance_transform.max(),255,0)
#sure_fg1 = 255.0-sure_fg
Fore_ground = np.uint8(Fore_ground)

plt.imshow(Fore_ground,cmap='gray')
#plt.title("Sure Foreground")
plt.axis('off')
plt.show();

unknown = cv2.subtract(Back_ground,Fore_ground)

ret2,markers = cv2.connectedComponents(Fore_ground)

markers = markers + 10


markers[unknown==255]=0

plt.imshow(markers,cmap='jet')
#plt.title("Markers")
plt.axis('off')
plt.show();


markers = cv2.watershed(img1,markers)
#the boundary region will be marked -1
#http://docs.opencv.org/3.3.1/d7/d1b//group-misc.html
#let us color oundaries in yellow. Opencv assigns boundaries to -1 after watershed

img1[markers==-1]=[0,255,255]
img2 = color.label2rgb(markers,bg_label=0)

plt.imshow(img)
#plt.title("Overlay on Original Image")
plt.axis('off')
plt.show();

plt.imshow(img2)
#plt.title("Infected Regions")
plt.axis('off')
plt.show();